In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

# ARIMA example
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.tsa.stattools as ts

# Datasets del challenge

In [2]:
d0t63 = pd.read_csv("positions_day_0_day_63.csv", encoding = 'utf8', error_bad_lines=False)

Todos los articulos vendidos del dia 0 al 63

In [3]:
d71 = pd.read_csv("positions_day_71.csv", encoding = 'utf8', error_bad_lines=False)

In [4]:
d71.head()

,date_number,product_id,category_id,position
0,71,65968023,2312968,6
1,71,70320877,2312968,24
2,71,70700940,2312968,22
3,71,76636979,2312968,1
4,71,83181798,2312968,0


Todos los articulos vendidos el dia 71

In [5]:
df = pd.read_csv("stock_and_sales_day_0_day_63.csv", encoding = 'utf8', error_bad_lines=False)

Ventas del dia 0 al 63

In [6]:
sr71t72 = pd.read_csv("stock_day_71_stock_replenishment_days_72-77.csv", encoding = 'utf8', error_bad_lines=False)

Reponer productos en la semana de 72 a 77

In [7]:
blocks = pd.read_csv("product_blocks.csv", encoding = 'utf8', error_bad_lines=False)

Bloques a apostar

In [8]:
products = pd.read_csv("products.csv", encoding = 'utf8', error_bad_lines=False)

Productos y precios del mismo

# Data navigation

In [9]:
df.head()

,date_number,product_id,color_id,size_id,sales,stock
0,0,310130,78467031,7026791,1,93
1,0,310130,78467031,8015276,2,62
2,0,310130,78467031,452052101,1,18
3,0,310130,78467031,541898038,2,42
4,0,310130,78467031,558685710,3,113


In [10]:
d0t63.drop(["category_id"], axis=1, inplace= True)

In [11]:
med = d0t63.groupby(["product_id", "date_number"], as_index=False).max()

In [12]:
med.rename(columns={"position": "max_position"}, inplace= True)

In [13]:
mini = d0t63.groupby(["product_id", "date_number"], as_index=False).min()

In [14]:
mini.rename(columns={"position": "min_position"}, inplace= True)

In [15]:
df = df.merge(blocks, on=['product_id'])

In [16]:
df = df.merge(products, on=['product_id'])

In [17]:
df = df.merge(mini, on=['product_id',"date_number"], how="left")

In [18]:
df = df.merge(med, on=['product_id',"date_number"], how="left")

In [19]:
df.head()

,date_number,product_id,color_id,size_id,sales,stock,block_id,family_id,subfamily_id,price,min_position,max_position
0,0,310130,78467031,7026791,1,93,1726,679611953,533441312,12.95,3.0,80.0
1,0,310130,78467031,8015276,2,62,1726,679611953,533441312,12.95,3.0,80.0
2,0,310130,78467031,452052101,1,18,1726,679611953,533441312,12.95,3.0,80.0
3,0,310130,78467031,541898038,2,42,1726,679611953,533441312,12.95,3.0,80.0
4,0,310130,78467031,558685710,3,113,1726,679611953,533441312,12.95,3.0,80.0


In [20]:
df.fillna(0, inplace=True)

In [21]:
df['date_number'].value_counts()

59    46636
60    46601
58    46028
61    45845
57    45687
63    45583
62    45246
56    44905
55    44384
53    44320
54    44278
45    43328
52    43293
46    43207
50    42712
49    42591
44    42581
43    42554
51    42493
47    42416
48    42101
42    41586
41    41254
39    41155
40    41040
38    40864
37    39956
36    39535
35    38720
32    38461
      ...  
29    37322
28    36567
27    36335
26    36147
25    36029
24    34864
22    34530
21    34494
23    34296
19    34230
18    34015
20    33942
17    33739
15    33129
16    32960
10    32796
14    32767
11    32415
13    32164
12    31880
9     30835
8     30826
7     30236
5     29831
4     29764
6     29577
3     29377
1     28810
2     28723
0     28105
Name: date_number, Length: 64, dtype: int64

In [22]:
def cambio(columna):
    dic = dict(df[columna].value_counts())
    value = 0
    for x in dic:
        value +=1
        dic[x] = value

    return df[columna].map(dic)

In [23]:
df["dia"] = df.apply(lambda row: row["date_number"] if row["date_number"] < 7 else row["date_number"] -7 * int(row["date_number"]/7), axis=1)

In [24]:
df.head()

,date_number,product_id,color_id,size_id,sales,stock,block_id,family_id,subfamily_id,price,min_position,max_position,dia
0,0,310130,78467031,7026791,1,93,1726,679611953,533441312,12.95,3.0,80.0,0.0
1,0,310130,78467031,8015276,2,62,1726,679611953,533441312,12.95,3.0,80.0,0.0
2,0,310130,78467031,452052101,1,18,1726,679611953,533441312,12.95,3.0,80.0,0.0
3,0,310130,78467031,541898038,2,42,1726,679611953,533441312,12.95,3.0,80.0,0.0
4,0,310130,78467031,558685710,3,113,1726,679611953,533441312,12.95,3.0,80.0,0.0


In [25]:
#df["subfamily_id"]=cambio('subfamily_id')
#df["size_id"]=cambio('size_id')
#df["family_id"]=cambio('family_id')
#df["product_id"]=cambio('product_id')
#df['color_id']=cambio('color_id')
#df['block_id']=cambio('block_id')

In [26]:
df['dia'].value_counts()

0.0    375554
4.0    345967
5.0    344090
6.0    342808
3.0    342317
2.0    335220
1.0    335105
Name: dia, dtype: int64

In [32]:
import sklearn
from sklearn import svm, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoLarsCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [28]:
df_train = df.loc[df["date_number"] < 57]
df_test = df.loc[df["date_number"] > 57]

X_train = df_train.drop("sales", axis=1).values
y_train = df_train["sales"].values

X_test = df_test.drop("sales", axis=1).values
y_test = df_test["sales"].values

In [43]:
clf = RandomForestRegressor(max_depth=20, random_state=0,n_estimators=100)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(f"Accuracy : {clf.score(X_test, y_test)*100}%\nMean absolute error: {mean_absolute_error(y_test, y_pred)}\nMean squared error: {mean_squared_error(y_test, y_pred)}")

TypeError: __init__() got an unexpected keyword argument 'shuffle'

In [30]:
df.shape

(2421061, 13)

In [37]:
df.head()

,date_number,product_id,color_id,size_id,sales,stock,block_id,family_id,subfamily_id,price,min_position,max_position,dia
0,0,310130,78467031,7026791,1,93,1726,679611953,533441312,12.95,3.0,80.0,0.0
1,0,310130,78467031,8015276,2,62,1726,679611953,533441312,12.95,3.0,80.0,0.0
2,0,310130,78467031,452052101,1,18,1726,679611953,533441312,12.95,3.0,80.0,0.0
3,0,310130,78467031,541898038,2,42,1726,679611953,533441312,12.95,3.0,80.0,0.0
4,0,310130,78467031,558685710,3,113,1726,679611953,533441312,12.95,3.0,80.0,0.0


In [38]:
 df.drop(columns=['dia'], inplace=True)

In [40]:
df.drop(columns=['color_id','block_id','product_id'], inplace=True)

In [41]:
df.head()

,date_number,size_id,sales,stock,family_id,subfamily_id,price,min_position,max_position
0,0,7026791,1,93,679611953,533441312,12.95,3.0,80.0
1,0,8015276,2,62,679611953,533441312,12.95,3.0,80.0
2,0,452052101,1,18,679611953,533441312,12.95,3.0,80.0
3,0,541898038,2,42,679611953,533441312,12.95,3.0,80.0
4,0,558685710,3,113,679611953,533441312,12.95,3.0,80.0
